In [19]:
import pygsti
from pygsti.circuits import Circuit
import random
from pygsti.modelpacks import smq1Q_XYZI as std
# Note that numpy 1.23 will break pygsti
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from forest.benchmarking.operator_tools import vec, unvec

In [20]:
model = std.target_model('CPTP')

In [122]:
b = n_qubit_pauli_basis(2)
b.labels[2]

'IY'

In [142]:
from forest.benchmarking.utils import *
from forest.benchmarking.operator_tools import *

class ErrorGen():
    """
    Implements the small Markovian taxonomy
    
    Parameters:
        -features: pauli transfer representations for different error processes
        -rates: sympy vector of rates
    """
    
    def __init__(self, num_qubits, label=('G', 0), parameterization='HSCA'):
        """
        setup the feature dictionary with keys that are sympy variables
        
        right now, I'm doing this by hand for a 1 qubit model, 
        TODO: implement logic to generalize to n-qubits
        """
        self.num_qubits = num_qubits
        self.basis = n_qubit_pauli_basis(num_qubits)
        self.feature_map = dict()
        gate_label = label[0]
        qbit_index = label[1]
        if 'H' in parameterization:
            # construct sympy symbols
            spy_symbols = ''
            for pstr in self.basis.labels:  
                spy_symbols += f'H^{{{pstr}}}_{{{gate_label}q{qbit_index}}} '
            self.hparams = sp.symbols(spy_symbols)
            # add the errors to the feature map
            for idx, pstr in enumerate(self.basis.labels):
                self.feature_map[self.hparams[idx]] = self.herr(pstr)
            
        else:
            self.hparams = []
            
        if 'S' in parameterization:
            # construct sympy symbols
            spy_symbols = ''
            for pstr in self.basis.labels:  
                spy_symbols += f'S^{{{pstr}}}_{{{gate_label}q{qbit_index}}} '
            self.sparams = sp.symbols(spy_symbols)
            # add the errors to the feature map
            for idx, pstr in enumerate(self.basis.labels):
                self.feature_map[self.sparams[idx]] = self.serr(pstr)

        else:
            self.sparams = []
            
        if 'C' in parameterization:
            spy_symbols = ''
            index_pairs = [(j, i) for i in range(1, len(self.basis.labels)) for j in range(1, i)]
            for idp in index_pairs:
                pstr1 = self.basis.labels[idp[0]]
                pstr2 = self.basis.labels[idp[1]]
                spy_symbols += f'C^{{{pstr1}.{pstr2}}}_{{{gate_label}q{qbit_index}}} '
            self.cparams = sp.symbols(spy_symbols)
            for n, idp in enumerate(index_pairs):
                pstr1 = self.basis.labels[idp[0]]
                pstr2 = self.basis.labels[idp[1]]
                self.feature_map[self.cparams[n]] = self.cerr(pstr1, pstr2)


        else:
            self.cparams = []
            
#         if 'A' in parameterization:
#             spy_symbols = ''
#             index_pairs = []
#             for i in range(1, len(self.basis.labels)):
#                 for j in range(1, i):
#                     index_pairs.append((i, j))
#             for idp in index_pairs:
#                 pstr1 = self.basis.labels[idp[1]]
#                 pstr2 = self.basis.labels[idp[0]]
#                 spy_symbols += f'A^{{{pstr1}.{pstr2}}}_{{{gate_label}q{qbit_index}}} '
#             self.aparams = sp.symbols(spy_symbols)
#             for n, idp in enumerate(index_pairs):
#                 pstr1 = self.basis.labels[idp[1]]
#                 pstr2 = self.basis.labels[idp[0]]
#                 self.feature_map[self.aparams[n]] = self.aerr(pstr1, pstr2)

#         else:
#             self.aparams = []
            
    @property
    def vec(self):
        return self.hparams+self.sparams+self.cparams+self.aparams
    
    @property
    def mat(self):
        mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype='O')
        for x in self.vec:
            mat += x*self.feature_map[x]
        return sp.Matrix(mat)
    
    @property
    def features(self):
        return [f for f in self.feature_map.values()]
    
    def herr(self, pauli_str):
        """
        hamiltonian error labelled by pauli
        
        Equation (10) in the paper
        """
        pindex = self.basis.labels.index(pauli_str)
        chi_mat = np.zeros((4**len(pauli_str), 4**len(pauli_str)), dtype=complex)
        chi_mat[0, pindex] = 1j
        chi_mat[pindex, 0] = -1j
        return chi2pauli_liouville(chi_mat)
    
    def serr(self, pauli_str):
        """
        stochatic error labelled by pauli
        
        Equation (14) in the paper
        """
        pindex = self.basis.labels.index(pauli_str)
        chi_mat = np.zeros((4**len(pauli_str), 4**len(pauli_str)), dtype=complex)
        chi_mat[0, 0] = -1
        chi_mat[pindex, pindex] = 1
        return chi2pauli_liouville(chi_mat)
    
    def cerr(self, pauli_str1, pauli_str2):
        """
        correlated stochatic error labelled by pair of paulis
        
        Equation (15) in the paper
        """
        pindex1 = self.basis.labels.index(pauli_str1)
        pindex2 = self.basis.labels.index(pauli_str2)
        
        # we also need product terms and their phases
        # this could probably be made more elegant
        pterm1 = str_to_pauli_term(pauli_str1)*str_to_pauli_term(pauli_str2)
        pterm2 = str_to_pauli_term(pauli_str2)*str_to_pauli_term(pauli_str1)
        
        phase1 = pterm1.coefficient
        phase2 = pterm2.coefficient
        pid1 = pterm1.id()
        pid2 = pterm1.id()
        # need to translate pyquil index convention to Pauli strings
        assert len(pauli_str1) == len(pauli_str2)
        pterm1 = ''
        flg = 0
        for i in range(len(pauli_str1)):
            if pid1[flg*2+1] == i:
                pterm1 += pid1[flg*2]
                flg += 1
            else:
                pterm1 += 'I'
        pterm2 = ''
        flg = 0
        for i in range(len(pauli_str2)):
            if pid2[flg*2+1] == i:
                pterm2 += pid1[flg*2]
                flg += 1
            else:
                pterm2 += 'I'
        prod_index1 = self.basis.labels.index(pterm1)
        prod_index2 = self.basis.labels.index(pterm2)

        chi_mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype=complex)
        chi_mat[pindex1, pindex2] += 1
        chi_mat[pindex2, pindex1] += 1
        # include the terms 
        # PQrII + QPrII + IIrPQ + IIrQP
        chi_mat[prod_index1, 0] += -0.5*phase1
        chi_mat[prod_index2, 0] += -0.5*phase2
        chi_mat[0, prod_index1] += -0.5*phase1
        chi_mat[0, prod_index2] += -0.5*phase2
        return chi2pauli_liouville(chi_mat)
    
    def aerr(self, pauli_str1, pauli_str2):
        """
        active error labelled by pair of paulis
        
        Equation (16) in the paper
        """
        pindex1 = self.basis.labels.index(pauli_str1)
        pindex2 = self.basis.labels.index(pauli_str2)
        
        # we also need product terms and their phases
        # this could probably be made more elegant
        pterm1 = str_to_pauli_term(pauli_str1)*str_to_pauli_term(pauli_str2)
        pterm2 = str_to_pauli_term(pauli_str2)*str_to_pauli_term(pauli_str1)
        
        phase1 = pterm1.coefficient
        phase2 = pterm2.coefficient
        pid1 = pterm1.id()
        pid2 = pterm1.id()
        # need to fix the id convention that pyquil uses
        # discard the even indices
        pid1 = [ele for idx, ele in enumerate(pid1) if idx % 2 == 0]
        pid2 = [ele for idx, ele in enumerate(pid2) if idx % 2 == 0]
        prod_index1 = self.basis.labels.index(''.join(pid1))
        prod_index2 = self.basis.labels.index(''.join(pid2))

        chi_mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype=complex)
        chi_mat[pindex1, pindex2] += 1j
        chi_mat[pindex2, pindex1] += -1j
        # include the terms 
        # PQrII - QPrII + IIrPQ - IIrQP
        chi_mat[prod_index1, 0] += 0.5j*phase1
        chi_mat[prod_index2, 0] += -0.5j*phase2
        chi_mat[0, prod_index1] += 0.5j*phase1
        chi_mat[0, prod_index2] += -0.5j*phase2
        return chi2pauli_liouville(chi_mat)

In [136]:
p = str_to_pauli_term('IX')*str_to_pauli_term('YI')
p.id()

'Y0X1'

In [162]:
len(egen.cparams)

105

In [161]:
egen = ErrorGen(2)
print(egen.cparams)
#egen.feature_map[egen.hparams[1]]
print(egen.feature_map[egen.cparams[1]])

(C^{IX.IY}_{Gq0}, C^{IX.IZ}_{Gq0}, C^{IY.IZ}_{Gq0}, C^{IX.XI}_{Gq0}, C^{IY.XI}_{Gq0}, C^{IZ.XI}_{Gq0}, C^{IX.XX}_{Gq0}, C^{IY.XX}_{Gq0}, C^{IZ.XX}_{Gq0}, C^{XI.XX}_{Gq0}, C^{IX.XY}_{Gq0}, C^{IY.XY}_{Gq0}, C^{IZ.XY}_{Gq0}, C^{XI.XY}_{Gq0}, C^{XX.XY}_{Gq0}, C^{IX.XZ}_{Gq0}, C^{IY.XZ}_{Gq0}, C^{IZ.XZ}_{Gq0}, C^{XI.XZ}_{Gq0}, C^{XX.XZ}_{Gq0}, C^{XY.XZ}_{Gq0}, C^{IX.YI}_{Gq0}, C^{IY.YI}_{Gq0}, C^{IZ.YI}_{Gq0}, C^{XI.YI}_{Gq0}, C^{XX.YI}_{Gq0}, C^{XY.YI}_{Gq0}, C^{XZ.YI}_{Gq0}, C^{IX.YX}_{Gq0}, C^{IY.YX}_{Gq0}, C^{IZ.YX}_{Gq0}, C^{XI.YX}_{Gq0}, C^{XX.YX}_{Gq0}, C^{XY.YX}_{Gq0}, C^{XZ.YX}_{Gq0}, C^{YI.YX}_{Gq0}, C^{IX.YY}_{Gq0}, C^{IY.YY}_{Gq0}, C^{IZ.YY}_{Gq0}, C^{XI.YY}_{Gq0}, C^{XX.YY}_{Gq0}, C^{XY.YY}_{Gq0}, C^{XZ.YY}_{Gq0}, C^{YI.YY}_{Gq0}, C^{YX.YY}_{Gq0}, C^{IX.YZ}_{Gq0}, C^{IY.YZ}_{Gq0}, C^{IZ.YZ}_{Gq0}, C^{XI.YZ}_{Gq0}, C^{XX.YZ}_{Gq0}, C^{XY.YZ}_{Gq0}, C^{XZ.YZ}_{Gq0}, C^{YI.YZ}_{Gq0}, C^{YX.YZ}_{Gq0}, C^{YY.YZ}_{Gq0}, C^{IX.ZI}_{Gq0}, C^{IY.ZI}_{Gq0}, C^{IZ.ZI}_{Gq0}, C^{XI.ZI}_{Gq

In [15]:
egen.vec

(H^{X0}_{G},
 H^{Y0}_{G},
 H^{Z0}_{G},
 S^{X0}_{G},
 S^{Y0}_{G},
 S^{Z0}_{G},
 C^{XY0}_{G},
 C^{YZ0}_{G},
 C^{ZX0}_{G},
 A^{XY0}_{G},
 A^{YZ0}_{G},
 A^{ZX0}_{G})

In [16]:
egen.mat

Matrix([
[               0,                                0,                                0,                                0],
[-4.0*A^{YZ0}_{G}, -2.0*S^{Y0}_{G} - 2.0*S^{Z0}_{G}, 2.0*C^{XY0}_{G} - 2.0*H^{Z0}_{G}, 2.0*C^{ZX0}_{G} + 2.0*H^{Y0}_{G}],
[-4.0*A^{ZX0}_{G}, 2.0*C^{XY0}_{G} + 2.0*H^{Z0}_{G}, -2.0*S^{X0}_{G} - 2.0*S^{Z0}_{G}, 2.0*C^{YZ0}_{G} - 2.0*H^{X0}_{G}],
[-4.0*A^{XY0}_{G}, 2.0*C^{ZX0}_{G} - 2.0*H^{Y0}_{G}, 2.0*C^{YZ0}_{G} + 2.0*H^{X0}_{G}, -2.0*S^{X0}_{G} - 2.0*S^{Y0}_{G}]])

In [17]:
"""print out all the elementary generators for 1 qubit egen"""
for x in egen.vec:
    print(x, '\n', egen.feature_map[x])

H^{X0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]
 [ 0.+0.j  0.+0.j  2.+0.j  0.+0.j]]
H^{Y0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  2.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]]
H^{Z0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
S^{X0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]]
S^{Y0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]]
S^{Z0}_{G} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
C^{XY0}_{G} 
 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j]


In [8]:
class GateSet():
    """
    Holds the information for
        -all native gates, states, and measurements
        -all error generator features (in a feature dict)
        -map between gate labels and parameter index list (index set)
        -map between parameter indices and error features (feature dict) 
        
    TODO: generalize to n-qubits
    """
    def __init__(self, model_pack, parameterization='HSCA'):
        # A list of error rates, each one corresponding to one feature in the above list
        self.rates = []
        self.feature_map = dict() # maps variables to their associated feature
        self.egen_from_label = dict() # maps gate labels to error generators
        self.target_model = model_pack.target_model('CPTP')
        self.gate_labels = model_pack.gates
        
        # Errors generators for all gates in the model
        for gate in model_pack.gates:
            if len(gate) <= 1:
                egen = ErrorGen(1, (gate, ''))
            else:
                egen = ErrorGen(1, gate)
            self.egen_from_label[gate] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
        # Errors for all measurements in the model
        for povm in model.povms.keys():
            egen = ErrorGen(1, (povm, 'M'))
            self.egen_from_label[povm] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
        # Errors for all state preparations in the model
        for prep in model.preps.keys():
            egen = ErrorGen(1, (prep, 'P'))
            self.egen_from_label[prep] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
            
    @property
    def vec(self):
        return self.rates
    
    def features_of(self, op_label):
        """Returns the list of error features associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].features
    
    def rates_of(self, op_label):
        """Returns the list of error rates associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].vec
    
    def error_generator_of(self, op_label):
        """Returns the error generator matrix associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].mat
    
    def target_op(self, op_label):
        """
        returns the target operation of the given operation
        """
        return self.target_model[op_label].to_dense()

    def native_meas(self):
        return [m.to_dense() for m in self.target_model['Mdefault'].values()]

    def rho0(self):
        return self.target_model['rho0'].to_dense()


In [9]:
gs.vec

NameError: name 'gs' is not defined

In [10]:
class CircuitAnalyzer:
    """Performs operations on a given pygsti Circuit object,
    which is part of a given GateSet."""
    def __init__(self, circuit, gateset):
        self.gateset = gateset
        self.circuit = circuit
        # Extracts the gates in the circuit as a list of labels
        # This may only work for sngle-qubit gates as written?
        self.gates = list(circuit.layertup)
        # Finds their PTMs (pauli transfer matrices)
        self.gate_matrices = [gateset.target_op(gate) for gate in self.gates]
        # Extracts the error matrices corresponding to each of these gates
        self.errors = [gateset.error_generator_of(gate) for gate in self.gates]
        
    def left_errors(self):
        """Moves all gate errors to the left of the circuit,
        and returns the resulting list of conjugated error generators.
        
        NOTE: In our Circuit object, the gates are ordered
        left-to-right from first-applied to last-applied.
        However, the error terms in the result of this function, are ordered
        left-to-right as they would be in matrix multiplication."""
        
        left_errors = []
        # Process the errors from the end of the list to its beginning
        # Because that's the order they'll appear in the resulting matrix mult
        n = len(self.errors)
        for i in range(n-1, -1, -1):
            error = self.errors[i]
            # Conjugate by all gates being applied after i: gates i+1 through n
            for j in range(i+1, n):
                gate_matrix = self.gate_matrices[j].round(10)
                error = gate_matrix * error * np.linalg.inv(gate_matrix)
            left_errors.append(error)
        return left_errors
    
    def linear_F_map(self):
        """Returns the linear approximation of the F map,
        after moving all errors to the left. This approximation assumes they commute.
        The result is a function of the error terms of the gate set.
        """
        # Simply take the sum of all error generators after moving them to the left
        left_errors = self.left_errors()
        return sum(left_errors[1:], left_errors[0])
    
    def order2_F_map(self):
        """Returns the order-2 approximation of the F map,
        after moving all errors to the left, and applying order-2 BCH.
        The result is a function of the error terms of the gate set.
        """
        left_errors = self.left_errors()
        BCH2 = lambda X, Y : X + Y + (X@Y - Y@X) / 2
        # After extracting the left-side errors,
        # reduce the list to one matrix by applying BCH to each pair
        result = left_errors[0]
        for F in left_errors[1:]:
            result = BCH2(result, F)
        return result

In [411]:
std.gates

[(), ('Gxpi2', 0), ('Gypi2', 0), ('Gzpi2', 0)]

In [412]:
gs = GateSet(std)

In [413]:
c.layertup

(Label(('Gxpi2', 0)), Label(('Gzpi2', 0)))

In [414]:
# Create a simple two gate circuit
c = Circuit([("Gxpi2", 0), ("Gzpi2", 0)])
print(c)

Qubit 0 ---|Gxpi2|-|Gzpi2|---



In [415]:
gs.error_generator_of(c[0])

Matrix([
[                   0,                                        0,                                        0,                                        0],
[-4.0*A^{YZ0}_{Gxpi2}, -2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} - 2.0*H^{Z0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} + 2.0*H^{Y0}_{Gxpi2}],
[-4.0*A^{ZX0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} + 2.0*H^{Z0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} - 2.0*H^{X0}_{Gxpi2}],
[-4.0*A^{XY0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{Y0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} + 2.0*H^{X0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gxpi2}]])

In [426]:
# Initialize our analyzer object. Verify that the error maps for both X(pi/2) gates are identical
a = CircuitAnalyzer(c, gs)
a.errors[1]

Matrix([
[                   0,                                        0,                                        0,                                        0],
[-4.0*A^{YZ0}_{Gzpi2}, -2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gzpi2}, 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{Y0}_{Gzpi2}],
[-4.0*A^{ZX0}_{Gzpi2}, 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} - 2.0*H^{X0}_{Gzpi2}],
[-4.0*A^{XY0}_{Gzpi2}, 2.0*C^{ZX0}_{Gzpi2} - 2.0*H^{Y0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*H^{X0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gzpi2}]])

In [442]:
a.linear_F_map()

Matrix([
[                                         0,                                                                                    0,                                                                                    0,                                                                                    0],
[-4.0*A^{YZ0}_{Gzpi2} + 4.0*A^{ZX0}_{Gxpi2},   -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gxpi2} - 2.0*H^{Z0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{X0}_{Gxpi2} + 2.0*H^{Y0}_{Gzpi2}],
[-4.0*A^{YZ0}_{Gxpi2} - 4.0*A^{ZX0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gxpi2} + 2.0*H^{Z0}_{Gzpi2},   -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2},  2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{X0}_{Gzpi2} + 2.0*H^{Y0}_{Gxpi2}],
[-4.0*A^{XY0}_{Gxpi2} - 4.0*A^{XY0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}

In [443]:
print(a.linear_F_map())

Matrix([[0, 0, 0, 0], [-4.0*A^{YZ0}_{Gzpi2} + 4.0*A^{ZX0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gxpi2} - 2.0*H^{Z0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{X0}_{Gxpi2} + 2.0*H^{Y0}_{Gzpi2}], [-4.0*A^{YZ0}_{Gxpi2} - 4.0*A^{ZX0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gxpi2} + 2.0*H^{Z0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{X0}_{Gzpi2} + 2.0*H^{Y0}_{Gxpi2}], [-4.0*A^{XY0}_{Gxpi2} - 4.0*A^{XY0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} - 2.0*H^{X0}_{Gxpi2} - 2.0*H^{Y0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} + 2.0*H^{X0}_{Gzpi2} - 2.0*H^{Y0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2}]])
